In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from env import host, username, password

# Acquire (acquire.py) - Zillow
For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.

# acquire & summarize

# 1. 
Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.
Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [2]:
def get_db_url(db_name):

    '''
    Connect to the SQL database with credentials stored in env file.
    Function parameter is the name of the database to connect to.
    Returns url.
    '''
    
    # Creates the url and the function returns this url
    url = f'mysql+pymysql://{username}:{password}@{host}/{db_name}'
    return (url)

In [18]:
def get_home_data():

    '''
    Connect to SQL Database with url function called within this function.
    Checks if database is already saved to computer in csv file.
    If no file found, saves to a csv file and assigns database to df variable.
    If file found, just assigns database to df variable.
    Returns df variable holding the  Home Value database.
    Includes all 52 Columns.
    '''
    
    # data_name allows the function to work no matter what a user might have saved their file name as
    # First, we check if the data is already stored in the computer
    # First conditional runs if the data is not already stored in the computer
    if os.path.isfile('zillow_home.csv') == False:

        # Querry selects the whole predicstion_2017 table from the database
        sql_querry = '''
                        SELECT *
                        FROM properties_2017
                        JOIN (SELECT id, logerror, pid, tdate FROM predictions_2017 AS pred_2017
                        JOIN (SELECT parcelid AS pid, Max(transactiondate) AS tdate FROM predictions_2017 GROUP BY parcelid) AS sq1
                        ON (pred_2017.parcelid = sq1.pid AND pred_2017.transactiondate = sq1.tdate)) AS sq2
                        ON (properties_2017.parcelid = sq2.pid)
                        LEFT JOIN airconditioningtype USING (airconditioningtypeid)
                        LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
                        LEFT JOIN buildingclasstype USING (buildingclasstypeid)
                        LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
                        LEFT JOIN propertylandusetype USING (propertylandusetypeid)
                        LEFT JOIN storytype USING (storytypeid)
                        LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
                        LEFT JOIN unique_properties USING (parcelid)
                        WHERE latitude IS NOT NULL AND longitude IS NOT NULL;
                    '''

        # Connecting to the data base and using the query above to select the data
        # the pandas read_sql function reads the query into a DataFrame
        df = pd.read_sql(sql_querry, get_db_url('zillow'))

        # If any duplicates found, this removes them
        # df.columns.duplicated() returns a boolean array, True for a duplicate or False if it is unique up to that point
        # Use ~ to flip the booleans and return the df as any columns that are not duplicated
        # df.loc accesses a group of rows and columns by label(s) or a boolean array
        df = df.loc[:,~df.columns.duplicated()]
        df = df.drop('pid',axis=1)

        # The pandas to_csv function writes the data frame to a csv file
        # This allows data to be stored locally for quicker exploration and manipulation
        df.to_csv('zillow_home.csv')

    # This conditional runs if the data has already been saved as a csv (if the function has already been run on your computer)
    else:
        # Reads the csv saved from above, and assigns to the df variable
        df = pd.read_csv('zillow_home.csv', index_col=0)

    return df


In [19]:
df = get_home_data()

In [20]:
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,censustractandblock,logerror,tdate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,...,6.059063e+13,0.025595,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,...,6.111001e+13,0.055619,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,...,6.059022e+13,0.005383,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,...,6.037300e+13,-0.103410,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,...,6.037124e+13,0.006940,2017-01-01,Central,None,None,Central,Condominium,None,None


# 2.
Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [22]:
print('Dataframe shape:',df.shape)

Dataframe shape: (77381, 68)


In [31]:
for col in df.columns:
    if col != 'parcelid' and col != 'id':
        print('Column name:',col)
        print(df[col].value_counts())
        print('\n')

Column name: typeconstructiontypeid
6.0     219
4.0       1
13.0      1
10.0      1
Name: typeconstructiontypeid, dtype: int64


Column name: storytypeid
7.0    50
Name: storytypeid, dtype: int64


Column name: propertylandusetypeid
261.0    52320
266.0    19294
246.0     2009
269.0     1944
248.0      727
247.0      535
265.0      333
263.0       74
275.0       58
260.0       37
267.0       29
31.0        15
264.0        6
Name: propertylandusetypeid, dtype: int64


Column name: heatingorsystemtypeid
2.0     33550
7.0     14065
24.0      846
6.0       776
20.0      104
13.0       59
18.0       23
1.0        13
10.0        3
11.0        1
Name: heatingorsystemtypeid, dtype: int64


Column name: buildingclasstypeid
4.0    14
3.0     1
Name: buildingclasstypeid, dtype: int64


Column name: architecturalstyletypeid
7.0     172
8.0      19
21.0      7
2.0       5
3.0       3
Name: architecturalstyletypeid, dtype: int64


Column name: airconditioningtypeid
1.0     23133
13.0     1574
5.0   

Central          23133
Yes               1574
None               167
Wall Unit           53
Refrigeration       26
Name: airconditioningdesc, dtype: int64


Column name: architecturalstyledesc
Contemporary     172
Conventional      19
Ranch/Rambler      7
Bungalow           5
Cape Cod           3
Name: architecturalstyledesc, dtype: int64


Column name: buildingclassdesc
Buildings having wood or wood and steel frames                                                                                                                                                                                             14
Buildings having exterior walls built of a non-combustible material such as brick, concrete, block or poured concrete. Interior partitions and roof structures are built of combustible materials. Floor may be concrete or wood frame.     1
Name: buildingclassdesc, dtype: int64


Column name: heatingorsystemdesc
Central       33550
Floor/Wall    14065
Yes             846
Forced air      7

In [36]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
parcelid,77381.0,1.300715e+07,3.481346e+06,1.071186e+07,1.153830e+07,1.253157e+07,1.421183e+07,1.676893e+08
typeconstructiontypeid,222.0,6.040541e+00,5.572847e-01,4.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,1.300000e+01
storytypeid,50.0,7.000000e+00,0.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00
propertylandusetypeid,77381.0,2.618263e+02,5.141199e+00,3.100000e+01,2.610000e+02,2.610000e+02,2.660000e+02,2.750000e+02
heatingorsystemtypeid,49440.0,3.920510e+00,3.592779e+00,1.000000e+00,2.000000e+00,2.000000e+00,7.000000e+00,2.400000e+01
buildingclasstypeid,15.0,3.933333e+00,2.581989e-01,3.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
architecturalstyletypeid,206.0,7.388350e+00,2.734542e+00,2.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,2.100000e+01
airconditioningtypeid,24953.0,1.813289e+00,2.967894e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.300000e+01
id,77381.0,1.495139e+06,8.609071e+05,3.490000e+02,7.520700e+05,1.497932e+06,2.240535e+06,2.982274e+06
basementsqft,50.0,6.797200e+02,6.897035e+02,3.800000e+01,2.730000e+02,5.150000e+02,7.965000e+02,3.560000e+03


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77381 non-null  int64  
 1   typeconstructiontypeid        222 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77381 non-null  float64
 4   heatingorsystemtypeid         49440 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      206 non-null    float64
 7   airconditioningtypeid         24953 non-null  float64
 8   id                            77381 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77381 non-null  float64
 11  bedroomcnt                    77381 non-null  float64
 12  buildingqualitytypeid         49672 non-null  float64
 13  c

# 3.
Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

#### What we want:   

|  column name  |  column Nan count  |  % rows with Nan  |

In [41]:
for col in df.columns:
    print('Columns name:',col)
    print('Nan count:',df[col].isnull().sum())
    print('\n')

Columns name: parcelid
Nan count: 0


Columns name: typeconstructiontypeid
Nan count: 77159


Columns name: storytypeid
Nan count: 77331


Columns name: propertylandusetypeid
Nan count: 0


Columns name: heatingorsystemtypeid
Nan count: 27941


Columns name: buildingclasstypeid
Nan count: 77366


Columns name: architecturalstyletypeid
Nan count: 77175


Columns name: airconditioningtypeid
Nan count: 52428


Columns name: id
Nan count: 0


Columns name: basementsqft
Nan count: 77331


Columns name: bathroomcnt
Nan count: 0


Columns name: bedroomcnt
Nan count: 0


Columns name: buildingqualitytypeid
Nan count: 27709


Columns name: calculatedbathnbr
Nan count: 609


Columns name: decktypeid
Nan count: 76767


Columns name: finishedfloor1squarefeet
Nan count: 71358


Columns name: calculatedfinishedsquarefeet
Nan count: 196


Columns name: finishedsquarefeet12
Nan count: 3632


Columns name: finishedsquarefeet13
Nan count: 77340


Columns name: finishedsquarefeet15
Nan count: 74372


Col

In [42]:
total = df.shape[0]
for col in df.columns:
    nulls = df[col].isnull().sum()
    print('Columns name:',col)
    print('Nan percentage:', (nulls / total) * 100)
    print('\n')

Columns name: parcelid
Nan percentage: 0.0


Columns name: typeconstructiontypeid
Nan percentage: 99.7131078688567


Columns name: storytypeid
Nan percentage: 99.93538465514791


Columns name: propertylandusetypeid
Nan percentage: 0.0


Columns name: heatingorsystemtypeid
Nan percentage: 36.108347010247996


Columns name: buildingclasstypeid
Nan percentage: 99.98061539654437


Columns name: architecturalstyletypeid
Nan percentage: 99.73378477920937


Columns name: airconditioningtypeid
Nan percentage: 67.75306599811323


Columns name: id
Nan percentage: 0.0


Columns name: basementsqft
Nan percentage: 99.93538465514791


Columns name: bathroomcnt
Nan percentage: 0.0


Columns name: bedroomcnt
Nan percentage: 0.0


Columns name: buildingqualitytypeid
Nan percentage: 35.80853181013427


Columns name: calculatedbathnbr
Nan percentage: 0.7870149002985228


Columns name: decktypeid
Nan percentage: 99.20652356521626


Columns name: finishedfloor1squarefeet
Nan percentage: 92.21643555911658



In [44]:
def null_counts(df):
    rows = df.shape()[0]
    nulls = pd.DataFrame(df.isnull().sum())
    nulls['null_percentage'] = (nulls / rows) * 100
    return nulls

In [46]:
null = null_counts(df)

TypeError: 'tuple' object is not callable

# 4. 
Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

# Prepare
# 1. 
Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

# 2. 
Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

The input:
- A dataframe
- A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
- A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).  

The output:
- The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
hint:
- Look up the dropna documentation.
- You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
- Make use of inplace, i.e. inplace=True/False.

# 3. 
Decide how to handle the remaining missing values:

- Fill with constant value.
- Impute with mean, median, mode.
- Drop row/column

# Zillow_Wrangle.py
Functions of the work above needed to acquire and prepare a new sample of data.

# Mall Customers

# 1. 
Acquire data from mall_customers.customers in mysql database.

# 2. 
Summarize data (include distributions and descriptive statistics).

# 3. 
Detect outliers using IQR.

# 4. 
Split data (train, validate, and test split).

# 5. 
Encode categorical columns using a one hot encoder.

# 6. 
Handles missing values.

# 7. 
Scaling

# wrangle_mall.py

# 1. 
Acquire data from mall_customers.customers in mysql database.

# 2. 
Split the data into train, validate, and split

# 3.
One-hot-encoding

# 4.
Missing values

# 5.
Scaling